In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pathlib
import sys

import jscatter
import numpy as np
import pandas as pd
import seaborn as sns

import embcomp as ec

data_dir = pathlib.Path.cwd() / ".." / "data" / "mair-2022-ismb"
name = "TISSUE_138_samples_FM96_OM138_035_CD45_live_fcs_110595"

raw = pd.read_parquet(data_dir / f"{name}_umap.parquet")
annotated = pd.read_parquet(data_dir / f"{name}_umap_annotated.parquet")
raw_knn_indices = ec.metrics.kneighbors(raw[["x", "y"]], k=16)
ann_knn_indices = ec.metrics.kneighbors(annotated[["x", "y"]], k=16)

In [8]:
import ipywidgets

import embcomp as ec

w = ec.widgets.pairwise(
    A=(raw[["x", "y"]], raw_knn_indices),
    B=(annotated[["x", "y"]], ann_knn_indices),
    labels=raw.complete_faust_label,
    robust=raw.cellType != "0_0_0_0_0",
)


w

GridBox(children=(HBox(children=(VBox(children=(Button(button_style='primary', icon='arrows', layout=Layout(wi…